En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [5]:
# imports
from q1_time import q1_time
from q1_memory import q1_memory
from q2_time import q2_time
from q2_memory import q2_memory
import cProfile
import subprocess

FILE_PATH = "../data/farmers-protest-tweets-2021-2-4.json"

## 1) Top 10 fechas com más tweets

### q1_time

Para esta función, utilicé DuckDB para leer y manipular los datos. Dado que el enfoque de esta función es el tiempo de procesamiento, DuckDB resulta ser una librería muy efectiva, ya que carga los datos en memoria y cuenta con un motor de ejecución vectorizado, lectores de archivos CSV optimizados, paralelismo automático y un planeamiento eficiente de consultas SQL. Esto facilitó la búsqueda de las fechas con más tuits y el usuario con más tuits por fecha, ya que los datos se manipularon en conjunto, sin necesidad de iterar sobre cada línea, utilizando consultas SQL tradicionales.

**Output**

In [6]:
q1_time(FILE_PATH)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Tiempo de ejecución

In [7]:
cProfile.run('q1_time(FILE_PATH)')

         4730 function calls (4726 primitive calls) in 0.010 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       26    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
       21    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1128(find_spec)
       84    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1222(__enter__)
       84    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1226(__exit__)
       21    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:124(setdefault)
       21    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap>:1240(_find_spec)
       21    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap>:1304(_find_and_load_unlocked)
       21    0.000    0.000    0.003    0.000 <frozen importlib._bootstrap>:1349(_find_and_load)
 

Uso de memoria

In [8]:
subprocess.run(['python3', '-m', 'memory_profiler', 'q1_time.py'])

Filename: q1_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     58.0 MiB     58.0 MiB           1   def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     8     69.1 MiB     11.2 MiB           1       con = duckdb.connect(database=':memory:')
     9   1361.9 MiB   1292.8 MiB           2       con.execute(f"""
    10                                                 CREATE TABLE tweets AS 
    11     69.1 MiB      0.0 MiB           1           SELECT * FROM read_ndjson('{file_path}')
    12                                             """)
    13   1364.8 MiB      2.9 MiB           1       con.execute("""
    14                                                 CREATE TABLE tweets_with_date AS
    15                                                 SELECT 
    16                                                     CAST(strptime(tweets['date'], '%Y-%m-%dT%H:%M:%S%z') AT TIME ZONE 'UTC' AS DATE) AS date_only,
    17                                   

CompletedProcess(args=['python3', '-m', 'memory_profiler', 'q1_time.py'], returncode=0)

### q1_memory

Dado que el objetivo es seleccionar las fechas y usuarios con más tuits, queda claro que el problema es de conteo. Por lo tanto, no es necesario que se carguen todos los datos en memoria para realizar los cálculos. Así fue posible optimizar el uso de memoria usando Python puro con generadores y contadores nativos. Los generadores garantizan un bajo uso de memoria, ya que esta se libera en cada iteración de datos. El conteo de tuits se realiza en tiempo de ejecución.

**Output**

In [11]:
q1_memory(FILE_PATH)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Tiempo de ejecución

In [10]:
cProfile.run('q1_memory(FILE_PATH)')

         1678156 function calls (1678150 primitive calls) in 1.794 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:263(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:312(__init__)
    49772    0.010    0.000    0.035    0.000 <frozen codecs>:322(decode)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
      2/1    0.002    0.001    1.730    1.730 <string>:1(<module>)
   117407    0.043    0.000    1.232    0.000 __init__.py:299(loads)
       13    0.000    0.000    0.000    0.000 __init__.py:599(__init__)
    51646    0.002    0.000    0.002    0.000 __init__.py:613(__missing__)
       10    0.000    0.000    0.001    0.000 __init__.py:622(most_common)
       13    0.000    0.000    0.000    0.000 __init__.py

Uso de memoria

In [12]:
subprocess.run(['python3', '-m', 'memory_profiler', 'q1_memory.py'])

Filename: q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8     52.2 MiB     52.2 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                         
    10     52.2 MiB      0.0 MiB           1       nested_dict = defaultdict(Counter)
    11     58.4 MiB      4.4 MiB      117408       for line in read_json_lines_generator(file_path):
    12     58.4 MiB      0.0 MiB      117407           date_part = datetime.fromisoformat(line['date']).date()
    13     58.4 MiB      1.7 MiB      117407           nested_dict[date_part][line['user']['username']] += 1
    14                                             
    15                                             # total tweet counts per date
    16     58.4 MiB      0.0 MiB          14       date_counts = {date: sum(user_counts.values()) for date, user_counts in nested_dict.items()}
    17                                             # top 10
    18   

CompletedProcess(args=['python3', '-m', 'memory_profiler', 'q1_memory.py'], returncode=0)

### Conclusión

Dado que este problema es de conteo, el enfoque de no cargar todos los datos en memoria es una buena opción. Así, si el bajo uso de memoria es un enfoque en el desarrollo, es recomendable explorar soluciones basadas en iteración. Con los datos de ejemplo, esta estrategia logró tanto un bajo uso de memoria como un buen tiempo de ejecución. Sin embargo, si la cantidad de archivos aumenta, esta estrategia podría no tener un buen desempeño.

Utilicé DuckDB para explorar esta herramienta, que es más nueva y, aunque cargue los datos en memoria, tiene distintas maneras de optimización que la hacen muy eficiente por sí sola, logrando un tiempo de ejecución de menos de 1 segundo.

| Enfoque | Abordaje | Tiempo(s)   | Memoria(MiB) |
| -------- | -------- | -------- | ------- |
| Tiempo | DuckDB   | < 1  | 1163 |
| Memoria | Generator | 1.7 | 56     |

---

## 2) Top 10 emojis

### q2_time

Para esta función, utilicé _Ujson_ en lugar de _json_ para cargar el archivo en memoria. _Ujson_ fue escrito principalmente en C, con un enfoque en el rendimiento de lectura y codificación. Por lo tanto, se espera una ganancia en la velocidad de lectura. Aunque los emojis sean representados con rangos de códigos Unicode y encontrados por expresiones regulares (regex), para garantizar que se consideren todos los emojis y sus variaciones, usé la librería _emoji_. El conteo de ocurrencias se hizo con el Counter nativo de Python.

**Output**

In [6]:
q2_time(FILE_PATH)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

Tiempo de ejecución

In [7]:
cProfile.run('q2_time(FILE_PATH)')

         85847528 function calls (85847522 primitive calls) in 15.184 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:263(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:312(__init__)
    49772    0.011    0.000    0.036    0.000 <frozen codecs>:322(decode)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
 17023302    1.624    0.000    2.752    0.000 <string>:1(<lambda>)
      2/1    0.035    0.017    9.474    9.474 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:599(__init__)
      870    0.000    0.000    0.000    0.000 __init__.py:613(__missing__)
        1    0.000    0.000    0.000    0.000 __init__.py:622(most_common)
        1    0.000    0.000    0.000    0.000 __init__.p

Uso de memoria

In [8]:
subprocess.run(['python3', '-m', 'memory_profiler', 'q2_time.py'])

Filename: q2_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8     56.9 MiB     56.9 MiB           1   def q2_time(file_path: str) -> List[Tuple[str, int]]:
     9     56.9 MiB      0.0 MiB           1       emoji_counter = Counter()
    10     62.0 MiB      0.0 MiB           2       with open(file_path, 'r', encoding='utf-8') as f:
    11     62.0 MiB      1.1 MiB      117408           for line in f:
    12     62.0 MiB      2.4 MiB      117407               tweet = ujson.loads(line)
    13     62.0 MiB      1.6 MiB      160329               for emoji_dict in emoji.emoji_list(tweet['content']):
    14     62.0 MiB      0.0 MiB       42922                   emoji_counter[emoji_dict['emoji']] += 1
    15     62.0 MiB      0.0 MiB           1       return emoji_counter.most_common(10)


[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


CompletedProcess(args=['python3', '-m', 'memory_profiler', 'q2_time.py'], returncode=0)

### q2_memory

En q2_memory se usó el mismo enfoque de _q1_time_, pero se empleó un generador para liberar la memoria en cada iteración de datos, logrando un poco más de efectividad en la asignación de memoria. Aunque en _q1_time_ el uso de un bucle _for_ funciona de manera similar a un generador, este enfoque garantiza la separación del ámbito de lectura y del procesamiento de datos. Además, el generador fue y puede ser usado en otras funciones.

**Output**

In [9]:
q2_memory(FILE_PATH)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

Tiempo de ejecución

In [10]:
cProfile.run('q2_memory(FILE_PATH)')

         87021602 function calls (87021596 primitive calls) in 15.926 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:263(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:312(__init__)
    49772    0.012    0.000    0.038    0.000 <frozen codecs>:322(decode)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
 17023302    1.663    0.000    2.845    0.000 <string>:1(<lambda>)
      2/1    0.000    0.000    7.531    7.531 <string>:1(<module>)
   117407    0.047    0.000    1.278    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:599(__init__)
      870    0.000    0.000    0.000    0.000 __init__.py:613(__missing__)
        1    0.000    0.000    0.000    0.000 __init__.py:622(

Uso de memoria

In [ ]:
subprocess.run(['python3', '-m', 'memory_profiler', 'q2_memory.py'])

### Conclusión

Este problema también puede ser abordado como un conteo. Sin embargo, para este conteo es necesario el análisis (parsing) de todos los tuits, lo que es una tarea computacionalmente más demandante.

La optimización de memoria se logró con el uso de generadores y bucles _for_. El tiempo de ejecución, aunque con poca diferencia, mejoró con el uso de _Ujson_ en lugar de _json_ para optimizar la carga de datos en memoria.

Posibles mejoras podrían explorarse usando motores o librerías expertas en análisis de textos y empleando multithreading o multiprocessing.


| Enfoque | Abordaje | Tiempo(s)   | Memoria(MiB) |
| -------- | -------- | -------- | ------- |
| Tiempo | ujson   | 15.1  | 62|
| Memoria | Generator | 15.9 | 61 |